# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import optuna

rng = np.random.RandomState(42)

# Data

In [2]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [3]:
model = AdaBoostRegressor(
    DecisionTreeRegressor(random_state=rng),
    random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 0.025505180605644315
Valid MAE: 3.216095890410959


In [4]:
def load_data():
    data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
    data = pd.read_csv(data_path)
    data['DATE'] = data['DATE'].astype('datetime64[ns]')
    X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
    y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
    X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
    y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
    return X_train, X_valid, y_train, y_valid

def objective(trial):
    PARAMS = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 250),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'estimator__max_depth': trial.suggest_int('estimator__max_depth', 1, 8),
        'estimator__min_samples_leaf': trial.suggest_int('estimator__min_samples_leaf', 1, 250),
        'estimator__min_impurity_decrease': trial.suggest_float('estimator__min_impurity_decrease', 0.0, 0.1),
        'estimator__ccp_alpha': trial.suggest_float('estimator__ccp_alpha', 0.0, 0.1)
    }
    X_train, _, y_train, _ = load_data()
    model = AdaBoostRegressor(
        DecisionTreeRegressor(random_state=rng),
        random_state=rng
    )
    model.set_params(**PARAMS)
    score = np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))
    return score

In [5]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 50, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials = 250,
    timeout = 3600*8, # in seconds
    n_jobs = -1,
    show_progress_bar = True
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-12-20 12:46:34,789] A new study created in RDB with name: no-name-30008d18-2c0e-441f-a5d0-ee97b8160d1b


  0%|          | 0/250 [00:00<?, ?it/s]

[I 2024-12-20 12:47:23,734] Trial 0 finished with value: -4.510809301921845 and parameters: {'n_estimators': 146, 'learning_rate': 0.6547297491011818, 'estimator__max_depth': 2, 'estimator__min_samples_leaf': 203, 'estimator__min_impurity_decrease': 0.0631019187743387, 'estimator__ccp_alpha': 0.00649120166132755}. Best is trial 0 with value: -4.510809301921845.
[I 2024-12-20 12:48:09,678] Trial 1 finished with value: -3.4135635443962116 and parameters: {'n_estimators': 100, 'learning_rate': 0.331532234045419, 'estimator__max_depth': 5, 'estimator__min_samples_leaf': 217, 'estimator__min_impurity_decrease': 0.002440786309735288, 'estimator__ccp_alpha': 0.035072327073986276}. Best is trial 1 with value: -3.4135635443962116.
[I 2024-12-20 12:48:15,951] Trial 3 finished with value: -3.844657716287184 and parameters: {'n_estimators': 167, 'learning_rate': 0.2547385149507741, 'estimator__max_depth': 3, 'estimator__min_samples_leaf': 57, 'estimator__min_impurity_decrease': 0.03713040534214695

In [6]:
print(f"Best score: {study.best_trial.value}")
print(f"Best params: {study.best_trial.params}")

Best score: -3.006806346898438
Best params: {'n_estimators': 48, 'learning_rate': 0.010713061675392342, 'estimator__max_depth': 8, 'estimator__min_samples_leaf': 38, 'estimator__min_impurity_decrease': 0.0016301462500532312, 'estimator__ccp_alpha': 0.007012665527843028}


In [7]:
model = AdaBoostRegressor(
    DecisionTreeRegressor(
        max_depth=8, min_samples_leaf=38,
        min_impurity_decrease=0.002, ccp_alpha=0.007,
        random_state=rng
    ),
    n_estimators=48, learning_rate=0.011,
    random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.8659197707059008
Valid MAE: 3.169976109551197


In [8]:
model = AdaBoostRegressor(
    DecisionTreeRegressor(
        max_depth=8, min_samples_leaf=50,
        #min_impurity_decrease=0.003, ccp_alpha=0.006,
        random_state=rng
    ),
    n_estimators=50, learning_rate=0.01,
    random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.8508248761150856
Valid MAE: 3.167783862042338
